In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash
from datetime import datetime
# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import Dash, dash_table, dcc, html, Input, Output, ctx, callback, callback_context
import plotly.express as px
from dash.dependencies import Input, Output, State
import base64
# Configure OS routines
import os
# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD import CRUD

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name
username = "username"
password = "password"

# Connect to database via CRUD Module
db = CRUD(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))
        
# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)
## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)
#FIXME Grazioso Salvare’s logo
image_filename = 'logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Center(html.B(html.H1('CS 340 Dashboard'))),
    html.Hr(),

    # Logo Top centered and scaled to 10%, linked to snhu.edu
    html.Div([  
        html.A(
            html.Center(html.Img(
                src='data:image/png;base64,{}'.format(encoded_image.decode()),
                id='logoImage',
                style={'width': '10%'}
            )),
            href='https://www.snhu.edu/'
        ),
    ]),

    # Format the radio buttons
    html.Div([
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water', 'value': 'water'},
                {'label': 'Mountain', 'value': 'mountain'},
                {'label': 'Disaster', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'},
            ],
            value='reset',
            labelStyle={'display': 'inline-block'}
        )
    ]),
    html.Hr(),

    # Format the data table 
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        style_data_conditional=[{'if': {'column_editable': False},'backgroundColor': 'white','color': 'rgb(30, 30, 30)'}],
        style_header_conditional=[{'if': {'column_editable': False},'backgroundColor': 'white','color': 'rgb(30, 30, 30)'}],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        derived_virtual_selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
       # derived_virtual_data=df.to_dict('records'),
        style_header={'border': '1px solid pink'},
    ),
    
    html.Br(),
    html.Hr(),
    
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        # scatter chart
        html.Div(
            id='graph-id',
            className='col s12 m6',
        ),
        # geolocation
        html.Div(
            id='map-id',
            className='col s12 m6',
        )
    ]),
    
    html.Hr(),
    # Developer name at the bottom
    html.Center(html.B(html.H1('Developed by Shera Adams'), style={'font-size': '10px'})),
])



#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(Output('datatable-id', 'data'),
              Output('datatable-id', 'columns'),
              [Input('filter-type', 'value')])
def update_dashboard(filterValue, **kwargs):
    # Filter data based on the selected type 
    if filterValue == 'mountain':
        # define a mountain query    
        mountain_rescue_query = {
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
        # save the df as result of the read function call passing the mountain query as a parameter
        df = pd.DataFrame.from_records(db.read(mountain_rescue_query))
    elif filterValue == 'water':
        # define a water query
        water_rescue_query = {
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
        # save the df as result of the read function call passing the water query as a parameter
        df = pd.DataFrame.from_records(db.read(water_rescue_query))
    elif filterValue == 'disaster':
        # define a disaster query
        disaster_query = {
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }
        # save the df as result of the read function call passing the disaster query as a parameter
        df = pd.DataFrame.from_records(db.read(disaster_query))
    else:
        # return all data if reset (default datatable/ read all data)
        df = pd.DataFrame.from_records(db.read({'animal_type': {'$in': ["Dog", "Cat", "Bird", "Other"]}}))
    
    # delete the _id collumn
    df.drop(columns=['_id'],inplace=True)   
    # define the column properties
    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    # create a dictionary from the data frame
    data = df.to_dict('records')
    #return a tuple of data, columns
    return (data, columns)

#############################################
# DATA CHART 
#############################################
# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('filter-type', 'value')]
)
def update_graphs(viewData, filterValue, **kwargs):
    # Create a data frame from the data
    dff = pd.DataFrame(viewData)

    if 'breed' in dff.columns:
        dff = dff['breed'].value_counts().reset_index()
            # define the column names as breed and count
        dff.columns = ['Breed', 'Count']

        # Combine the breeds with less than 70 members to make the chart more readable
        if filterValue == 'reset':
            dff.loc[dff['Count'] < 70, 'Breed'] = 'Other Breeds'

        # Create a bubble chart
        bubble_fig = px.scatter(
            dff,
            x='Breed',  # X-axis variable
            y='Count',  # Y-axis variable
            size='Count',  # Size of the bubbles
            color='Breed',  # Color of the bubbles
            title='Preferred Animals by Breed',  # title the chart
            template='plotly_white'  # set the template style
        )

        # optional: set labels to diagonal 
        # bubble_fig.update_layout(xaxis_tickangle=45)

        # Return the graph
        return [dcc.Graph(
            figure=bubble_fig,
            # set the size of the graph
            style={'width': '900px', 'height': '500px'}
        )]
    else:
        return []



#############################################
# update row style
#############################################
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_rows')]  # check
)
def update_styles(selected_rows, **kwargs):
    styles = [{
        'if': { 'row_index': i },  # Use 'row_index' to target rows
        'background_color': '#D2F3FF'
    } for i in selected_rows]

#############################################
# Geolocation and marker
#############################################
# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index, **kwargs):  
    # handle the case that the viewData is empty
    if viewData is None or not viewData:
        return []

    # Check for empty index
    if index is None or not index:
        return []
    
    # set the index default to the first row
    else: 
        row = index[0]  
    # define the data fram
    dff = pd.DataFrame.from_dict(viewData)
    # return the map with a marker at the index's longitude/ longitude coordinates
    return [     # Austin TX is at [30.75,-97.48]
        dl.Map(style={'width': '900px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),  
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]

app.run_server(debug=True, port=8085)


connection successful
Dash app running on http://127.0.0.1:8085/
